# Exercício Prático
**1. Coletar os seguintes dados da página: https://books.toscrape.com**

**Catálogo:**
- Classics

- Science Fiction

- Humor

- Business

**Coletar os seguintes dados de cada livro:**
- Nome do livro

- Preço em libras

- Avaliação dos consumidores

- Disponível em estoque

**2. Faça um plano escrito para cada uma das perguntas de negócio, contendo:**
- Saída: A simulação da tabela e gráfico final.

- Processo: A sequência de passos organizada pela lógica de execução

- Entrada: O link para as fontes de dados.


# Planejamento para solução (SAPE)


### Saida
**Solução resposta**
- Coletar os dados referente a todas as categorias

**Formato da entrega**
- Tabela em formato .csv

**Local da entrega**
- Repositório Git

### Processo

**Construção da resposta**
1. Realizar coleta atavés do pacote BeatifulSoup:
2. Reconhecer a estrutura do site e html
3. Verificar referência no html que serão válidas para todas as coletas
4. Coletar o page size para coletar todos os dados
5. Coletar URLs de todos os books
6. Criar um Loop For para coletar todos os dados solicitados: Nome do livro, Preço em libras, Avaliação dos consumidores, Disponível em estoque e Categoria do livro
7. Gerar um DataFrame dos dados coletados com Pandas
8. Criar coluna com dia e hórario do scrapy, através do datetime.
9. Exportar dados para .csv

**Formato da Entrega**
- Tabela, com as seguintes colunas:
| book_name | book_category | book_price | book_rating | book_stock | book_id | scrapy_datetime |

**Local de Entrega**
- Jupyter Notebook

### Entrada

**Fonte de dados**
- https://books.toscrape.com/

**Ferramentas**
- Python 3.8.5
- BeautifulSoup
- Jupyter Notebook

# 0.0 Import Libs

In [25]:
import pandas as pd
import numpy as np
import requests
import re

from datetime import datetime
from bs4 import BeautifulSoup

# 1.0 Create path and headers

In [ ]:
url = 'https://books.toscrape.com/'
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5),AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'}
page = requests.get(url, headers=headers)

# BeautifulSoup Objetct
soup = BeautifulSoup(page.text, 'html.parser')

## 1.1 All URL Books

In [ ]:
#================== Get all url books ====================#
# page size
n_page = soup.find('form', class_='form-horizontal')
n_page = n_page.get_text().split(' ')
n_page = n_page[0].split('\n')[3]
n_page = int(n_page) / 20
page_list = list(np.arange(1, int(n_page)+1,1))

list_url = []

for i in range(len(page_list)):
    url = 'https://books.toscrape.com/catalogue/page-'+ str(page_list[i]) +'.html' 
    page = requests.get(url, headers=headers)
    soup = BeautifulSoup(page.text, 'html.parser')
    product_url = soup.find_all('div',class_='image_container')
    l = [p.find('a').get('href') for p in product_url]
    list_url = list_url + l 

# 2.0 Get Books Details

In [ ]:
#================== Get books details ====================#
#Nome do livro
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5),AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'}
rating_list = ['One','Two','Three','Four','Five']
cols = ['book_name','book_category','book_price','book_rating','book_stock','book_id']
books = pd.DataFrame(columns=cols)

for i in range(len(list_url)):
    url = 'https://books.toscrape.com/catalogue/' + list_url[i]
    page = requests.get(url, headers=headers)
    soup = BeautifulSoup(page.text, 'html.parser')

    # product name
    product_details = soup.find('div', class_='col-sm-6 product_main')
    name = str(product_details.h1.string)

    # product price
    product_details = soup.find('p', class_='price_color')
    price = product_details.get_text()[1:]

    #product rating
    for p in range(len(rating_list)):
        product_details = soup.find('div', class_='col-sm-6 product_main')

        if product_details.find('p', class_='star-rating '+ str(rating_list[p])):
            aux = product_details.find('p', class_='star-rating '+ str(rating_list[p]))
            rating = aux['class'][1]
        else:
            next

    # product stock
    product_details = soup.find('p', class_='instock availability')
    stock = product_details.get_text().strip()
    
    # product category
    product_details = soup.find('ul', class_='breadcrumb')
    category = list(filter(None, product_details.get_text().split('\n')))[2]
    
    # create dataframe
    aux = pd.DataFrame([name,category,price,rating,stock]).T
    aux.columns = ['book_name','book_category','book_price','book_rating','book_stock']
    aux['book_id'] = i
    
    books = pd.concat([books, aux], axis=0)

# reset index
books = books.reset_index().drop('index',axis=1)

# create datetime scrap

books['scrapy-datetime'] = datetime.now().strftime('%Y-%m-%d %H:%M:%S')

In [ ]:
# Export to csv
books.to_csv('books-list-complete.csv', index=False )

# Clean Data

In [109]:
data = pd.read_csv('books-list-complete.csv')

# book_name
# book_category
data['book_category'] = data['book_category'].apply(lambda x: x.replace(' ','_').lower() if pd.notnull(x) else x )

# book_price
data['book_price'] = data['book_price'].apply(lambda x: x.replace('£','')).astype(float)

# book_rating
data['book_rating'] = data['book_rating'].apply(lambda x: 1 if x == 'One' else
                                                          2 if x == 'Two' else
                                                          3 if x == 'Three' else
                                                          4 if x == 'Four' else 5)

# book_stock
regex = '(\d+)'
data['book_stock'] = data['book_stock'].apply(lambda x: re.search(regex, x).group(0) if pd.notnull(x) else x)

# book_id
# scrapy-datetime

# book category add_a_comment to Default
data['book_category'] = data['book_category'].apply(lambda x: 'default' if x == 'add_a_comment' else x)

# book category sorted
data = data.sort_values('book_category', ascending=True)

In [111]:
# export to csv data cleaned
data.to_csv('books-data-clean.csv', index=False )